In [1]:
import os
import torch
import numpy as np
from pgd import PGD
from dknn import AdvDKNN, DKNN
from models.cnn import CNN
from torchvision.datasets import MNIST

In [2]:
root = "./datasets"
model_weights = "./model_weights/mnist_cnn_at.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
download = not os.path.exists(os.path.join(root,"MNIST"))
random_seed = 1234
class_samp_size = 2000

trainset = MNIST(root=root, train=True, download=download)
testset = MNIST(root=root, train=False, download=download)

np.random.seed(random_seed)
subset = [
    np.random.choice(
        np.where(np.array(trainset.targets)==i)[0],
        size=class_samp_size,
        replace=False
    ) for i in range(10)
]
train_data = torch.cat([
    torch.FloatTensor(trainset.data[inds].unsqueeze(1)/255) for inds in subset
], dim=0)
train_targets = torch.arange(10).repeat_interleave(class_samp_size)

model = CNN()
model.load_state_dict(torch.load(model_weights, map_location=device))
model.eval()
model.to(device)

CNN(
  (f1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(8, 8), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (f2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(6, 6), stride=(2, 2))
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (f3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [3]:
pgd = PGD(eps=80 / 255., step_size=20 / 255., max_iter=10, random_init=True, batch_size=128)

x, y = (
    torch.FloatTensor(testset.data.unsqueeze(1) / 255.)[:1000],
    torch.LongTensor(testset.targets)[:1000]
)

x_adv = pgd.generate(model, x, y, device=device)

In [4]:
dknn = DKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,],
    device=device
)
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,],
    device=device,
    eps=60 / 255.,
    step_size=20 / 255.,
    max_iter=10,
    random_init=True
)

pred_benign_dknn = dknn(x.to(device)).argmax(axis=1)
acc_benign_dknn = (pred_benign_dknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by DKNN is {acc_benign_dknn}")

pred_adv_dknn = dknn(x_adv.to(device)).argmax(axis=1)
acc_adv_dknn = (pred_adv_dknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by DKNN is {acc_adv_dknn}")

pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


The benign accuracy by DKNN is 0.99
The adversarial accuracy by DKNN is 0.588
The benign accuracy by Adv-DKNN is 0.992
The adversarial accuracy by Adv-DKNN is 0.6


In [5]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,],
    device=device,
    eps=40 / 255.,
    step_size=20 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:00<00:00,  2.45it/s]


The benign accuracy by Adv-DKNN is 0.99
The adversarial accuracy by Adv-DKNN is 0.596


In [6]:
dknn = DKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[1,2],
    device=device
)
pred_benign_dknn = dknn(x.to(device)).argmax(axis=1)
acc_benign_dknn = (pred_benign_dknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by DKNN is {acc_benign_dknn}")

pred_adv_dknn = dknn(x_adv.to(device)).argmax(axis=1)
acc_adv_dknn = (pred_adv_dknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by DKNN is {acc_adv_dknn}")

100%|██████████| 2/2 [00:13<00:00,  6.57s/it]


The benign accuracy by DKNN is 0.985
The adversarial accuracy by DKNN is 0.683


In [7]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[1,2],
    device=device,
    eps=60 / 255.,
    step_size=20 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 2/2 [00:11<00:00,  5.98s/it]


The benign accuracy by Adv-DKNN is 0.983
The adversarial accuracy by Adv-DKNN is 0.697


In [8]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,],
    weight=[0,0],
    device=device,
    eps=60 / 255.,
    step_size=20 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


The benign accuracy by Adv-DKNN is 0.986
The adversarial accuracy by Adv-DKNN is 0.598


In [9]:
advdknn = AdvDKNN(
    model,
    train_data,
    train_targets,
    hidden_layers=[2,],
    weight=[0,0],
    device=device,
    eps=40 / 255.,
    step_size=20 / 255.,
    max_iter=10,
    random_init=True
)
pred_benign_advdknn = advdknn(x.to(device)).argmax(axis=1)
acc_benign_advdknn = (pred_benign_advdknn == y.numpy()).astype("float").mean()
print(f"The benign accuracy by Adv-DKNN is {acc_benign_advdknn}")

pred_adv_advdknn = advdknn(x_adv.to(device)).argmax(axis=1)
acc_adv_advdknn = (pred_adv_advdknn == y.numpy()).astype("float").mean()
print(f"The adversarial accuracy by Adv-DKNN is {acc_adv_advdknn}")

100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


The benign accuracy by Adv-DKNN is 0.989
The adversarial accuracy by Adv-DKNN is 0.602
